### Get mangroves for all years for AOI and calc extent changes for canopy cover classes

In [ ]:
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import os
from datacube import Datacube
from datacube.utils import geometry
import rioxarray
from datacube.utils.cog import write_cog
dc = datacube.Datacube(app="cyclone mangroves")

pd.set_option('display.max_columns', None)


### fill in data (doing one cyclone at a time for the moment)
option to export area of canopy cover classes as csv
option to export geotiff of before and after cyclone mangrove dataset

In [ ]:
# cyclone_name = 'Yasi'
# cyclone_time = 2011

# cyclone_name = 'Larry'
# cyclone_time = 2006

# cyclone_name = 'Ita'
# cyclone_time = 2014

# cyclone_name = 'Marcia'
# cyclone_time = 2015

# cyclone_name = 'Lam'
# cyclone_time = 2015

# cyclone_name = 'George'
# cyclone_time = 2007

# cyclone_name = 'Ingrid_Landfall1'
# cyclone_time = 2005

# cyclone_name = 'Ingrid_Landfall2'
# cyclone_time = 2005

# cyclone_name = 'Ingrid_Landfall3'
# cyclone_time = 2005

# cyclone_name = 'Monica_Landfall1'
# cyclone_time = 2006

# cyclone_name = 'Monica_Landfall2'
# cyclone_time = 2006

# cyclone_name = 'Nathan_Landfall1'
# cyclone_time = 2015

# cyclone_name = 'Nathan_Landfall2'
# cyclone_time = 2015

cyclone_name = 'Laurence'
cyclone_time = 2009

export_as_csv = True

export_before_after_cyclone_mangrove_dataset = True

### load in geotiff

In [ ]:
geotiff_path = './FINALlocalWind/'+cyclone_name+'_30m_EPSG3577.tif'
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio(geotiff_path)
# change -9999 to 0 #### not sure this needs to be done, -9999 should be considered 'not cyclone'?
geotiff_da = xr.where(geotiff_da == -9999, 0, geotiff_da.values)
# Covert our xarray.DataArray into a xarray.Dataset
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
dataset = geotiff_ds.rename({1: 'wind_speed'})# get extent of cyclone dataset
geobox = dataset.extent

### load mangrove data on extent of geotiff

In [ ]:
# get extent of cyclone dataset
geobox = dataset.extent

In [ ]:
# 1987 - 2021 datasets
mangrove_datasets = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                            geopolygon=dataset.extent, 
                            output_crs='EPSG:3577', 
#                             time = ('2010-01-01', '2011-12-31'),
                            resolution=(-30, 30))

### create dataframe and append areas (km2 - multiple each pixel by 0.0009) for each annual mangrove dataset

##### 0 = not observed/missing data
##### 1 = 20-50% canopy cover
##### 2 = 50-80% canopy cover
##### 3 = 80-100% canopy cover
##### 255 =  no mangroves


In [ ]:
# set up df with rows for each canopy_cover_class
df = pd.DataFrame([0, 1, 2, 3, 255])

# for each time
for t in mangrove_datasets.canopy_cover_class:
    # get year as string
    time = str(t.time.values)[:4]
    # get canopy_cover_class and pixel counts
    unique, counts = np.unique(t, return_counts=True)
    item = np.asarray((unique, counts)).T
    # check values looks correct
#     print(item)
    # append to df (divide counts by 0.0009 to give area in km2)
    df[time] = pd.Series(counts*0.0009)

In [ ]:
df

#### export

In [ ]:
# export as csv?
if export_as_csv == True:
    df.to_csv('./'+cyclone_name+'_area_km2.csv')
else:
    pass

In [ ]:
# export before and after cyclone datasets
if export_before_after_cyclone_mangrove_dataset == True:
    # get 1 year before and year of cyclone data
    time_of_cyclone = (str(cyclone_time - 1) + '-01-01', str(cyclone_time) + '-12-31')

    # before and after cyclone datasets
    mangrove_datasets = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                                geopolygon=dataset.extent, 
                                output_crs='EPSG:3577', 
                                time = time_of_cyclone,
                                resolution=(-30, 30))
    # for time in canopy_cover_class
    for t in mangrove_datasets.canopy_cover_class:
        # get time as string for filesave
        time = str(t.time.values)[:4]
        print(time)
        write_cog(geo_im=t,
                  fname='./'+cyclone_name+'_mangrove_canopy_cover_'+time+'.tif',
                  overwrite=True)
else:
    pass